In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
foundations_url = 'https://www.sephora.com/shop/foundation-makeup'

In [3]:
grids = []

for pagenum in range(1,21):
    page = BeautifulSoup(requests.get((foundations_url+'?currentPage={}&pageSize=12').format(pagenum)).content, 'html.parser')
    grids.append(page.find('div', {'data-comp':'ProductGrid'}))
    

In [20]:
foundation_info = pd.DataFrame(columns=['foundation_name', 'link', 'Brand', 'price', 'colors', 'rank'])

In [21]:
#Get the
for grid in grids:
    for row in grid.find_all('div', {'class':"css-dkxsdo"}):
        for product in row.find_all('a', {'class':"css-ix8km1"}):
            if product:
                foundation_name = product.find('span',{'data-at':'sku_item_name'}).text

                rank = product.find('div', {'class':"css-1pjru6n"}).attrs['style']

                try:
                    colors = product.find('div', {'class':'css-rrjz1n'}).text
                except:
                     colors = None
                space_index = product.attrs['href'].index(' ')

                try:
                    price = product.find('span', {'class':'css-0'}).text
                except:
                    price = product.find('span', {'class':'css-1qdampm'}).text

                brand = product.find('span', {'class':'css-ktoumz OneLinkNoTx'}).text

                link = ('https://www.sephora.com' + product.attrs['href'][:space_index])

                foundation_info.loc[len(foundation_info)] = [foundation_name, link, brand, price, colors, rank]
            

In [22]:
len(foundation_info)


230

In [26]:
foundation_info.head()

,foundation_name,link,Brand,price,colors,rank
0,Pro Filt'r Soft Matte Longwear Foundation,https://www.sephora.com/product/pro-filtr-soft...,FENTY BEAUTY by Rihanna,$34.00,49 more colors,width:81.986%
1,Double Wear Stay-in-Place Foundation,https://www.sephora.com/product/double-wear-st...,Estée Lauder,$42.00,55 more colors,width:88.848%
2,Ultra HD Invisible Cover Foundation,https://www.sephora.com/product/ultra-hd-invis...,MAKE UP FOR EVER,$19.00 - $43.00,40 more colors,width:78.78999999999999%
3,Luminous Silk Foundation,https://www.sephora.com/product/luminous-silk-...,Giorgio Armani Beauty,$42.00 - $64.00,43 more colors,width:85.74199999999999%
4,Your Skin But Better™ CC+™ Cream with SPF 50+,https://www.sephora.com/product/your-skin-but-...,IT Cosmetics,$15.00 - $38.00,13 more colors,width:81.32%


In [27]:
foundation_info.to_csv('Foundation_info.csv')

In [18]:
fondation_info['rank'] = fondation_info['rank'].str.replace('width:', '')
fondation_info['rank'] = fondation_info['rank'].str.replace('%', '')